## Advent of code 2022 day 21-25
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import functools
import re
import copy
import math
import sys
import time
import json
import heapq
import bisect
import random
import sortedcontainers
#import cProfile

In [ ]:
# utility functions and version check

def get_line_groups(lines, nostrip=False):
    '''return list of lists of lines, each separated by empty lines, ignores empty lines from start and end,
    by default also strips all lines (if nostrip is set only strips empty lines)'''
    lines=list(lines)
    lines.append('') # add terminator
    res=[]
    group=[]
    for line in lines:
        line_str=line.strip()
        if nostrip==False or len(line_str)<1:
            line=line_str
        if len(line)>0:
            group.append(line)
        elif len(group)>0: # close group
            res.append(group)
            group=[]
    return res

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def exit():
    raise StopExecution()
    
print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

In [ ]:
# 2022 day 24
# mv ~/Downloads/input* data_src/2022-day-24-input.txt
# big input file looks like: a map
# idea: part 1 parse into board size, start and endpos, and list of blizzards,
# then generate subsequent maps and BFS (BFS twist here is maxwait and explored,
# maxwait so suboptimal todo entries are allowed to a certain point, explored to avoid 
# repeating the same todo over and over)

sample2='''
#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#
'''

sample3='''
#.#####
#.....#
#>....#
#.....#
#...v.#
#.....#
#####.#
'''

def parse_board(lines):
    bw=len(lines[0])
    bh=len(lines)
    i=lines[0].index('.')
    assert i==1
    startpos=(i, 0)
    i=lines[bh-1].index('.')
    assert i==bw-2
    endpos=(i, bh-1)
    bliz=[]
    for y,row in enumerate(lines):
        for x,c in enumerate(row):
            if c!='.' and c!='#':
                assert c in {'<', '>', '^', 'v'}
                bliz.append( (x,y,c) )
    return bliz, startpos, endpos, bw, bh

def get_bliset(bliz):
    '''input is list of (x,y,c), return set of (x,y)'''
    return {(tup[0], tup[1]) for tup in bliz}

def get_bliz_turn(blizmap, turn, bw, bh, startpos, endpos):
    if turn not in blizmap:
        get_bliz_turn(blizmap, turn-1, bw, bh, startpos, endpos)
        bliz=blizmap[turn-1][0]
        bliz2=[]
        for bl in bliz:
            x,y,c=bl
            if c=='>':
                x+=1
                if x>=bw-1:
                    x=1
            elif c=='<':
                x-=1
                if x<=0:
                    x=bw-2
            elif c=='v':
                y+=1
                assert (x,y)!=endpos
                if y>=bh-1:
                    y=1
            elif c=='^':
                y-=1
                assert (x,y)!=startpos
                if y<=0:
                    y=bh-2
            else:
                assert False
            bliz2.append( (x,y,c) )
        bliset2=get_bliset(bliz2)
        blizmap[turn]=(bliz2, bliset2)
    return blizmap[turn][1]

def do_walk(blizmap, startpos, endpos, bw, bh, startturn):
    maxwait=100
    todos=set() # each todo is (x,y,turn)
    todos.add( (startpos[0], startpos[1], startturn) )
    explored=set() # all todos collected
    explored.add( (startpos[0], startpos[1], startturn) )
    reached={} # maps (x,y) to min. turn to reach
    reached[startpos]=startturn
    while len(todos)>0:
        pos=todos.pop()
        x,y,turn=pos
        blisetnext=get_bliz_turn(blizmap, turn+1, bw, bh, startpos, endpos)
        for x2,y2 in [(x+1,y), (x-1,y), (x,y+1), (x,y-1), (x,y)]:
            if x2<=0 or x2>=bw-1:
                continue
            if y2<=0 or y2>=bh-1:
                if (x2,y2)!=startpos and (x2,y2)!=endpos:
                    continue
            if (x2,y2) in blisetnext:
                continue
            newpos=(x2,y2)
            if newpos not in reached or reached[newpos]>turn+1:
                reached[newpos]=turn+1
            if turn+1<reached[newpos]+maxwait and (x2, y2, turn+1) not in explored:
                todos.add( (x2, y2, turn+1) )
                explored.add( (x2, y2, turn+1) )
    return reached[endpos]

sample1=open('data_src/2022-day-24-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
bliz, startpos, endpos, bw, bh=parse_board(lines)
print(f'{startpos=}, {endpos=}, {bw=}, {bh=}')
#print(bliz)
blizmap={} # maps turn to (bliz,bliset)
blizmap[0]=(bliz,get_bliset(bliz))

# part 1
#for i in range(6):
#    bliset=get_bliz_turn(blizmap, i, bw, bh, startpos, endpos)
#    print(f'at turn {i}: {bliset}')
turn=do_walk(blizmap, startpos, endpos, bw, bh, 0)
print(f'part 1: {turn}, max explored: {max(blizmap.keys())}')

# part 2
turn2=do_walk(blizmap, endpos, startpos, bw, bh, turn)
turn3=do_walk(blizmap, startpos, endpos, bw, bh, turn2)
print(f'part 2: back in {turn2-turn}, again in {turn3-turn2}, total {turn3}')

# part 1: 238, max explored: 5690
# part 2: back in 271, again in 242, total 751

In [ ]:
# 2022 day 23
# mv ~/Downloads/input* data_src/2022-day-23-input.txt
# big input file looks like: a map
# idea: part 1 parse into set, then execute each round using dict and counter

sample2='''
....#..
..###.#
#...#.#
.#...##
#.###..
##.#.##
.#..#..
'''

sample3='''
.....
..##.
..#..
.....
..##.
.....
'''

def create_board(lines):
    board=set() # (x,y) of each elf
    for y, row in enumerate(lines):
        for x, c in enumerate(row):
            if c=='#':
                board.add( (x,y) )
    return board

def print_board(board):
    y_vals={tup[1] for tup in board }
    x_vals={tup[0] for tup in board }
    res=[]
    for y in range(min(y_vals), max(y_vals)+1):
        row=''
        for x in range(min(x_vals), max(x_vals)+1):
            c='#' if (x,y) in board else '.'
            row+=c
        print(row)

def count_empty_in_bbox(board):
    y_vals={tup[1] for tup in board }
    x_vals={tup[0] for tup in board }
    cemp=0
    for y in range(min(y_vals), max(y_vals)+1):
        for x in range(min(x_vals), max(x_vals)+1):
            if (x,y) not in board:
                cemp+=1
    return cemp

def do_round1(board, roundi):
    '''returns count of moved elves'''
    # part 1: plan
    directions=['N', 'S', 'W', 'E']
    plan={} # maps (x,y) of elf to new position
    counter=collections.Counter() # per new position counts elves
    for pos in board: # per elf
        x,y=pos
        countaround=0
        for x2 in range(x-1, x+2):
            for y2 in range(y-1, y+2):
                if (x2,y2) in board:
                    countaround+=1
        if countaround<=1: # nobody around but ourselves
            continue
        for di in range(4): # per direction
            direc=directions[(roundi+di)%len(directions)]
            countdirec=0
            if direc=='N':
                for x2 in range(x-1, x+2):
                    if (x2,y-1) in board:
                        countdirec+=1
                newpos=(x, y-1)
            elif direc=='S':
                for x2 in range(x-1, x+2):
                    if (x2,y+1) in board:
                        countdirec+=1
                newpos=(x,y+1)
            elif direc=='W':
                for y2 in range(y-1, y+2):
                    if (x-1,y2) in board:
                        countdirec+=1
                newpos=(x-1, y)
            elif direc=='E':
                for y2 in range(y-1, y+2):
                    if (x+1,y2) in board:
                        countdirec+=1
                newpos=(x+1, y)
            else:
                assert False
            if countdirec==0:
                plan[pos]=newpos
                counter[newpos]+=1
                #print(f'round {roundi} move plan from {pos} to {newpos} with direc. {direc}, newpos counter {counter[newpos]}')
                break # done with that elf
    # part 2: execute
    countmoves=0
    for pos in list(board):
        newpos=plan.get(pos)
        if newpos is None:
            continue
        if counter[newpos]>1:
            continue
        board.remove(pos)
        board.add(newpos)
        countmoves+=1
    return countmoves

sample1=open('data_src/2022-day-23-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
board=create_board(lines)

# part 1
#print_board(board)
for roundi in range(10):
    cround=do_round1(board, roundi)
    #print(f'after moving {cround} elves in round {roundi}:')
    #print_board(board)
count1=count_empty_in_bbox(board)
print(f'part 1: {count1}')

# part 2
board=create_board(lines)
roundi=0
while True:
    cround=do_round1(board, roundi)
    roundi+=1
    if cround<1:
        break
print(f'part 2: {roundi}')

# part 1: 3970
# part 2: 923

In [ ]:
# 2022 day 22
# mv ~/Downloads/input* data_src/2022-day-22-input.txt
# big input file looks like: big ass map
# idea: part 1 parse, then execute

sample2='''
        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5
'''

def pad_board(board):
    n=max([len(s) for s in board])
    for i, row in enumerate(board):
        while len(row)<n:
            row+=' '
        board[i]=row
    return board

def do_walk1(board, walk, x, y, direc):
    assert board[y][x]=='.'
    for w in walk:
        if w=='L':
            direc=(direc-1)%4
        elif w=='R':
            direc=(direc+1)%4
        else:
            assert isinstance(w, int)
            for _ in range(w):
                nx=x
                ny=y
                if direc==0:
                    nx+=1
                    if nx>=len(board[ny]) or board[ny][nx]==' ':
                        nx=0
                        while board[ny][nx]==' ':
                            nx+=1
                elif direc==1:
                    ny+=1
                    if ny>=len(board) or board[ny][nx]==' ':
                        ny=0
                        while board[ny][nx]==' ':
                            ny+=1
                elif direc==2:
                    nx-=1
                    if nx<0 or board[ny][nx]==' ':
                        nx=len(board[ny])-1
                        while board[ny][nx]==' ':
                            nx-=1
                elif direc==3:
                    ny-=1
                    if ny<0 or board[ny][nx]==' ':
                        ny=len(board)-1
                        while board[ny][nx]==' ':
                            ny-=1
                else:
                    assert False
                if board[ny][nx]=='#':
                    break
                x=nx
                y=ny
                assert board[y][x]=='.'
    return x,y,direc

def do_walk(board, walk, do_part=1, faces=None, facesz=None, wraps=None, boardcopy=None):
    # find start
    y=0 # coords are 0-based
    x=board[0].index('.')
    direc=0 # right=0, 1=down, 2=left, 3=up
    print(f'start: {x=}, {y=}, {direc=}')
    # execute walk
    if do_part==1:
        endpos=do_walk1(board, walk, x, y, direc)
    else:
        assert do_part==2
        endpos=do_walk2(board, walk, x, y, direc, faces, facesz, wraps, boardcopy) # run next cell to define
    # passwd
    x,y,direc=endpos
    print(f'end: {x=}, {y=}, {direc=}')
    res=1000*(y+1)+4*(x+1)+direc
    return res

sample1=open('data_src/2022-day-22-input.txt').read()
groups=get_line_groups(sample1.splitlines(), nostrip=True)
assert len(groups)==2
assert len(groups[1])==1
walk=re.split(r'([RL])', groups[1][0])
walk=[(s if s in {'L', 'R'} else int(s)) for s in walk]

# part 1
board=pad_board(list(groups[0]))
passwd=do_walk(board, walk, do_part=1)
print(f'part 1: {passwd}')

# part 2
board=pad_board(list(groups[0]))
# sample2
'''
faces=[[0, 0, 1, 0], [2, 3, 4, 0], [0, 0, 5, 6]]
facesz=4
wraps={ # connected edges and rotations
    '1A': ('2A', 180),
    '1B': ('6B', 180),
    '1D': ('3A', -90),
    '2A': ('1A', 180),
    '2C': ('5C', 180),
    '2D': ('6C', -90),
    '3A': ('1D', 90),
    '3C': ('5D', -90),
    '4B': ('6A', 90),
    '5C': ('2C', 180),
    '5D': ('3C', 90),
    '6A': ('4B', -90),
    '6B': ('1B', 180),
    '6C': ('2D', 90),
}
boardcopy=list(groups[0])
'''
# sample1
faces=[[0, 5, 6], [0, 4, 0], [2, 3, 0], [1, 0, 0]]
facesz=50
wraps={
    '1B': ('3C', -90),
    '1C': ('6A', 0),
    '1D': ('5A', -90),
    '2A': ('4D', 90),
    '2D': ('5D', 180),
    '3B': ('6B', 180),
    '3C': ('1B', 90),
    '4B': ('6C', -90),
    '4D': ('2A', -90),
    '5A': ('1D', 90),
    '5D': ('2D', 180),
    '6A': ('1C', 0),
    '6B': ('3B', 180),
    '6C': ('4B', 90),
}
boardcopy=None
passwd=do_walk(board, walk, do_part=2, faces=faces, facesz=facesz, wraps=wraps, boardcopy=boardcopy)
if boardcopy is not None:
    for line in boardcopy:
        print(line)
print(f'part 2: {passwd}')

# (for sample2 the answers are 6032 and 5031)
# part 1: 136054
# part 2: 122153

In [ ]:
# part 2 extra functions (run first before the cell above)

def bc_mark(x, y, bc, boardcopy):
    if boardcopy is None:
        return
    boardcopy[y]=boardcopy[y][:x]+bc+boardcopy[y][x+1:]
    bc=chr(ord(bc)+1)
    if bc>'9':
        bc='0'
    return bc

def do_walk2(board, walk, x, y, direc, faces, facesz, wraps, boardcopy):
    assert board[y][x]=='.'
    bc=bc_mark(x, y, '0', boardcopy)
    for w in walk:
        if w=='L':
            direc=(direc-1)%4
        elif w=='R':
            direc=(direc+1)%4
        else:
            assert isinstance(w, int)
            for _ in range(w):
                nx=x
                ny=y
                ndirec=direc
                if direc==0:
                    nx+=1
                    if nx>=len(board[ny]) or board[ny][nx]==' ':
                        nx,ny,ndirec=do_wrap(nx-1, ny, ndirec, faces, facesz, wraps)
                elif direc==1:
                    ny+=1
                    if ny>=len(board) or board[ny][nx]==' ':
                        nx,ny,ndirec=do_wrap(nx, ny-1, ndirec, faces, facesz, wraps)
                elif direc==2:
                    nx-=1
                    if nx<0 or board[ny][nx]==' ':
                        nx,ny,ndirec=do_wrap(nx+1, ny, ndirec, faces, facesz, wraps)
                elif direc==3:
                    ny-=1
                    if ny<0 or board[ny][nx]==' ':
                        nx,ny,ndirec=do_wrap(nx, ny+1, ndirec, faces, facesz, wraps)
                else:
                    assert False
                if board[ny][nx]=='#':
                    break
                x=nx
                y=ny
                direc=ndirec
                assert board[y][x]=='.'
                bc=bc_mark(x, y, bc, boardcopy)
    return x,y,direc

def do_wrap(x, y, direc, faces, facesz, wraps):
    '''from position on edge wrap around to new edge, crossing it by 1 step'''
    # find current face and edge
    fx0=x // facesz
    fy0=y // facesz
    curface=faces[fy0][fx0]
    curedge={0: 'B', 1: 'C', 2: 'D', 3: 'A'}[direc] # direc right=0, 1=down, 2=left, 3=up
    #print(f'{curface=}, {curedge=}')
    # find position along edge from top or left side
    along=(x % facesz) if curedge in {'A', 'C'} else (y % facesz)
    #print(f'{along=}')
    # from new face, edge, rotation determine new position and direction
    newtup=wraps[str(curface)+curedge]
    newedge,rotation=newtup
    newface=int(newedge[0])
    newedge=newedge[1]
    assert 'A' <= newedge <= 'D'
    #print(f'{newface=}, {newedge=}, {rotation=}')
    facefound=False
    for fy1 in range(len(faces)):
        for fx1 in range(len(faces[0])):
            if faces[fy1][fx1]==newface:
                facefound=True
                break
        if facefound:
            break
    # set nx,ny to topleft of new edge
    nx=fx1*facesz
    ny=fy1*facesz # for A and D this is already correct
    if newedge=='B':
        nx+=facesz-1
    elif newedge=='C':
        ny+=facesz-1
    #print(f'new edge topleft {nx=}, {ny=}, {facesz=}')
    # update using along, set ndirec
    if rotation==0:
        if newedge in {'A', 'C'}:
            nx+=along
        else:
            ny+=along
        ndirec=direc
    elif rotation==90:
        if newedge=='A':
            nx+=(facesz-1-along)
        elif newedge=='B':
            ny+=along
        elif newedge=='C':
            nx+=(facesz-1-along)
        elif newedge=='D':
            ny+=along
        ndirec=(direc+1)%4
    elif rotation== -90:
        if newedge=='A':
            nx+=along
        elif newedge=='B':
            ny+=(facesz-1-along)
        elif newedge=='C':
            nx+=along
        elif newedge=='D':
            ny+=(facesz-1-along)
        ndirec=(direc-1)%4
    elif rotation==180:
        if newedge in {'A', 'C'}:
            nx+=(facesz-1-along)
        else:
            ny+=(facesz-1-along)
        ndirec=(direc+2)%4
    else:
        assert False
    return nx,ny,ndirec

# do_wrap tests for sample2
#assert do_wrap(11, 5, 0, faces, facesz, wraps)==(14, 8, 1)
#assert do_wrap(10, 11, 1, faces, facesz, wraps)==(1, 7, 3)

In [ ]:
# 2022 day 21
# mv ~/Downloads/input* data_src/2022-day-21-input.txt
# big input file looks like: 2597 rules
# idea: part 1 parse as tuples, then DFS
# part 2: take the two root parts, one varies with humn, the other doesn't,
# the variable part can be increasing or decreasing with humn, binary search to find humn

sample2='''
root: pppw + sjmn
dbpl: 5
cczh: sllz + lgvd
zczc: 2
ptdq: humn - dvpt
dvpt: 3
lfqf: 4
humn: 5
ljgn: 2
sjmn: drzm * dbpl
sllz: 4
pppw: cczh / lfqf
lgvd: ljgn * ptdq
drzm: hmdt - zczc
hmdt: 32
'''

def calc(nodename, rules):
    tup=rules[nodename]
    if len(tup)==1:
        return tup[0]
    assert len(tup)==3
    arg1=calc(tup[0], rules)
    arg2=calc(tup[2], rules)
    if tup[1]=='+':
        return arg1+arg2
    elif tup[1]=='-':
        return arg1-arg2
    elif tup[1]=='*':
        return arg1*arg2
    elif tup[1]=='/':
        return arg1/arg2
    else:
        assert False

sample1=open('data_src/2022-day-21-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ s.split() for s in lines ]
for tup in data:
    if tup[0].endswith(':'):
        tup[0]=tup[0][:-1]
    if len(tup)==2:
        tup[1]=int(tup[1])
data={ tup[0]: tup[1:] for tup in data }
# part 1
res=int(calc('root', data))
print(f'part 1: {res}')
print()

# part 2
# find parts of root
root1=data['root'][0]
root2=data['root'][2]
# try with different humn to see which is variable
data['humn']=[0]
root1a=calc(root1, data)
root2a=calc(root2, data)
data['humn']=[1]
root1b=calc(root1, data)
root2b=calc(root2, data)
assert (root1a==root1b) != (root2a==root2b) # could be tricky if both were variable..
if root1a==root1b:
    fixedroot=root1
    varroot=root2
    increasing= root2b>root2a
else:
    fixedroot=root2
    varroot=root1
    increasing= root1b>root1a
targetval=calc(fixedroot, data)
print(f'part 2: fixedroot={targetval}, {increasing=}')
# binary search with ever-increasing range
found=False
start0= -1000
while not found:
    start=start0
    end= -start
    while start<end-1:
        mid=(start+end)//2
        data['humn']=[mid]
        val=calc(varroot, data)
        if increasing:
            if val<targetval:
                start=mid
            else:
                end=mid
        else:
            if val<targetval:
                end=mid
            else:
                start=mid
    found=False
    for i in [start, end]:
        data['humn']=[i]
        val=calc(varroot, data)
        if int(val)==int(targetval):
            print(f'part 2: for humn={i} varroot={int(val)}')
            found=True
    if not found:
        start0*=1000

# part 1: 223971851179174
# part 2: 3379022190351

In [ ]:
# part 2 alternative solution: recursively reversing the tree

def check_fixed(tree, hnode, rules):
    '''does this tree have a fixed value (returns val) or depend on hnode value (returns None) ?'''
    rules[hnode]=[0]
    val=calc(tree, rules)
    rules[hnode]=[1]
    if calc(tree, rules)==val:
        return val
    else:
        return None

def solve(tree, targetval, hnode, rules):
    if tree==hnode:
        return targetval
    assert len(rules[tree])==3
    t1=rules[tree][0]
    oper=rules[tree][1]
    t2=rules[tree][2]
    if (v1:=check_fixed(t1, hnode, rules))!=None:
        if oper=='+':
            return solve(t2, targetval-v1, hnode, rules) # v1+?=targetval
        elif oper=='-':
            return solve(t2, v1-targetval, hnode, rules) # v1-?=targetval
        elif oper=='*':
            return solve(t2, targetval/v1, hnode, rules) # v1*?=targetval
        elif oper=='/':
            return solve(t2, v1/targetval, hnode, rules) # v1/?=targetval
    else:
        assert (v2:=check_fixed(t2, hnode, rules))!=None
        if oper=='+':
            return solve(t1, targetval-v2, hnode, rules) # ?+v2=targetval
        elif oper=='-':
            return solve(t1, targetval+v2, hnode, rules) # ?-v2=targetval
        elif oper=='*':
            return solve(t1, targetval/v2, hnode, rules) # ?*v2=targetval
        elif oper=='/':
            return solve(t1, targetval*v2, hnode, rules) # ?/v2=targetval
    assert False

hnode='humn'
t1=data['root'][0]
t2=data['root'][2]
if (v1:=check_fixed(t1, hnode, data))!=None:
    res=solve(t2, v1, hnode, data)
else:
    assert (v2:=check_fixed(t2, hnode, data))!=None
    res=solve(t1, v2, hnode, data)
res=int(res)
print(f'part 2: {res}')


In [ ]:
# TEMPLATE
# 2022 day 21
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2022-day-21-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2022-day-21-input.txt').read()
lines=[s for s in sample2.splitlines() if len(s)>0 ]
groups=get_line_groups(sample2.splitlines(), nostrip=False)
data=[ int(s) for s in lines[0].split(',') ]
data=[ s.split() for s in lines ]
data=[ [cmd, int(num), 0] for cmd, num in data ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed